In [17]:
import os
import pandas as pd
from datetime import datetime
import shutil

In [18]:
def validation(file_path):
    try:
        # Attempt to read the CSV file at the specified file path into a DataFrame
        orders_df = pd.read_csv(file_path,header=0)
    except pd.errors.EmptyDataError:
        # If an EmptyDataError occurs during the attempt to read the file,
        # create a DataFrame containing an error message indicating an empty file
        err_df = pd.DataFrame({'error_file': ['Empty File Found']})
        # Return the DataFrame with the error message
        return err_df
    
     # check if dataframe is empty
    if orders_df.empty:
        # If the DataFrame is empty, add a new column 'reject_reason' with an error message
        orders_df['reject_reason']= ['No Data Found']
        # Return the DataFrame with the added column and error message
        return orders_df
        
    #Extract all column names from the DataFrame 'orders_df' and convert them into a list
    selected_columns=list(orders_df.columns)
    
    #3-the order date should not be in future
    # Get the current date in the 'YYYY-MM-DD' format
    current_date = datetime.now().strftime('%Y-%m-%d')
    # Check if there are any order dates in the 'orders_df' DataFrame that are greater than the current date
    if (orders_df['order_date'] > current_date).any():
        # If there are, initialize the 'reject_reason' column with an empty string
        orders_df['reject_reason']=''
        # Create a DataFrame 'inv_datedf' containing rows where 'order_date' is greater than the current date
        inv_datedf=orders_df[orders_df['order_date'] > current_date]
        # Update the 'reject_reason' column for rows in 'inv_datedf' by appending the specified error message
        orders_df.loc[inv_datedf, 'reject_reason'] += 'product_id not present; '
        
        
    # 4- any field should not be empty
    # Check if there are any null values in the selected columns of the 'orders_df' DataFrame
    if orders_df[selected_columns].isnull().any().any():
        # Check if the 'reject_reason' column is not already present in the DataFrame
        if 'reject_reason' not in orders_df.columns:
            # If not present, create the 'reject_reason' column with empty strings
            orders_df['reject_reason']=''
        # Update the 'reject_reason' column for rows with null values in the selected columns
        orders_df.loc[orders_df[selected_columns].isnull().any(axis=1), 'reject_reason'] += 'empty field; '
        
    #5- the orders should be from Mumbai or Bangalore only.
    valid_cities = ['Mumbai', 'Bangalore']
    invalid_city_rows = ~orders_df['city'].isin(valid_cities)
    if  invalid_city_rows.any():
         if 'reject_reason' not in orders_df.columns:
            orders_df['reject_reason']=''
         invalid_city=orders_df[invalid_city_rows]
         orders_df.loc[invalid_city, 'reject_reason'] += 'invalid city; '
        
    #here we read master_file
    product="C:\\Users\\hp\\project1\\NamasteKart\\master_data"
    # Read all CSV files in the specified directory
    file = [file for file in os.listdir(product) if file.endswith('.csv')]
    # Iterate through each file in the list
    for f in file:
        # Construct the full file path by joining the directory path and the file name
        file_path1 = os.path.join(product, f)
        product_master=pd.read_csv(file_path1)
       
    df_merge = pd.merge(orders_df,product_master, how='left', left_on='product_id', right_on='product_id')
   
    #do the following validations for each order
    #1- product id should be present in product master table   
    null_order_rows = df_merge['product_id'].isnull()
    if null_order_rows.any():
        if 'reject_reason' not in df_merge.columns:
            df_merge['reject_reason']=''
        p_orders=df_merge[null_order_rows]
        df_merge.loc[p_orders, 'reject_reason'] += 'product_id not present; '
        
    #2- total sales amount should be (product price from product master table * quantity)
    df_merge['sales_validation'] = df_merge['price'] * df_merge['quantity']
    invalid_sales_rows = df_merge['sales'] != df_merge['sales_validation'] 
    invalid_sal=df_merge[invalid_sales_rows]
  
    
    if not invalid_sal.empty:
         if 'reject_reason' not in df_merge.columns:
            df_merge['reject_reason']=''
         df_merge.loc[invalid_sales_rows,'reject_reason'] += 'Invalid total sales amount; '
      
        
    # Check if the 'reject_reason' column is present in the 'df_merge' DataFrame and is not empty
    if 'reject_reason' in df_merge.columns and not df_merge['reject_reason'].empty:
        # Append 'reject_reason' to the list of selected columns
        selected_columns.append('reject_reason')
        # Update 'df_merge' DataFrame to include only the selected columns
        df_merge=df_merge[selected_columns]
        # Filter 'df_merge' DataFrame to include only rows where 'reject_reason' is not empty
        df_merge=df_merge[df_merge['reject_reason']!='']
        
        
        return df_merge
    else:
        return None

In [19]:
def return_valid():
    incoming_files_cnt,rejected_files_cnt,success_files_cnt=0,0,0
    current_date = datetime.now().strftime('%Y%m%d')
    incoming_files = f'NamasteKart/incoming_files/{current_date}/'
    success_files = f'NamasteKart/success_files/{current_date}/'
    rejected_files = f'NamasteKart/rejected_files/{current_date}/'
    os.makedirs(incoming_files, exist_ok=True)
    os.makedirs(success_files, exist_ok=True)
    os.makedirs(rejected_files, exist_ok=True)
    
    for file_name in os.listdir(incoming_files):
         incoming_files_cnt+=1
         file_path = os.path.join(incoming_files, file_name)
         validation_result = validation(file_path)
         if validation_result is not None:
          
        
             rejected_files_cnt+=1
             error_file_name = f"error_{file_name}"
             # Save the validation result DataFrame to a CSV file with the error file name in the rejected_files directory
             validation_result.to_csv(os.path.join(rejected_files, error_file_name), index=False)
             # build the file path for the original file in the rejected_files directory
             rejected_file_path= os.path.join(rejected_files, file_name)
             # Copy the original file to the directory where rejected files are stored
             shutil.copy(file_path,rejected_file_path)
         else:
              # Create a file path for the original file in the success_files directory          
              success_file_path= os.path.join(success_files, file_name)
              # copy the original file to the directory where sucess files are stored.                               
              shutil.copy(file_path,success_file_path)                     
              success_files_cnt+=1
        
       
    
        
    return  incoming_files_cnt,rejected_files_cnt,success_files_cnt   


In [20]:
incoming_files_cnt,rejected_files_cnt,success_files_cnt=return_valid()

In [21]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import datetime

def send_email(incoming_files_cnt,rejected_files_cnt,success_files_cnt):
    # Email configurations
    sender_email = "nidhikumari29sep@gmail.com"  # Replace with your email
    sender_password = "upxf muwr bhzb gneo"  # Replace with your email password
    recipient_email = "nidhinansi@gmail.com"
    today_date=datetime.date.today()
    subject = f"Validation Email {today_date}"

    # Email content
    body = f"Total {incoming_files_cnt} incoming files, {success_files_cnt} successful files, and {rejected_files_cnt} rejected files for {today_date}."

    # Create the MIME object
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = recipient_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    # Establish the connection with the SMTP server (in this case, Gmail)
    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, recipient_email, msg.as_string())


send_email(incoming_files_cnt,rejected_files_cnt,success_files_cnt)